<a href="https://colab.research.google.com/github/diarrabell/fashion-recs/blob/main/fashion_recs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/diarrabell/fashion-recs.git

fatal: destination path 'fashion-recs' already exists and is not an empty directory.


In [10]:
import pandas as pd
import numpy as np